### IR Book 16.3 Concept Code

This code implements the Bernoulli EM algorithm as described in the IR Book, example 16.3. This exercise is useful to ensure an understanding of the algorithm and to check functionality. There are a few bits that aren't explained will in the book but become apparent when working with this code. 

- During the computation of the $r_{n,k}$ classifications it is best to use log(probabilities) in order to prevent underflow

- The use of $\epsilon$ as a smoothing parameter is crucial to the good behavior of the algorithm and to avoid either a divide by zero or a log(0) problem. In the IR book $\epsilon$ is set to 0.0001. Setting to smaller values causes the algorithm to take more iterations to converge to the same solution as in the book.

In a Bernoulli Mixture Model a document is a vector of Booleans indicating the presence of a term.

The conditional probability of a document given a set of parameters is given by:

$$P(d \;|\;\theta) = \sum_{k=1}^{K}\alpha_k(\prod_{tm \in d} q_{mk})(\prod_{tm \notin d}(1 - q_mk))$$

This is the sum for each class of the product of the probabilities of the terms in a document with 1 minus the probabilities of the terms not in the document.

The probability that a document from cluster $\omega_{k}$ containts term $t_{m}$ is given by:

$$q_{mk} = P(U_{m} = 1 \; | \;\omega_{k})$$

The prior $\alpha_{k}$ of cluster $\omega_{k}$ is the probability document $d$ is in $\omega_{k}$ if we have no other information about it.

When we don't know the classifications of the documents we can use Expectation Maximization iteratively to arrive at classifications, $r_{nk}$.

**E Step**

$$\tag{1} r_{nk} = \frac{\alpha_{k}(\prod_{tm \in d_n}q_{mk})(\prod_{tm \in d_n}1-q_{mk})}{\sum_{k=1}^{K}\alpha_{k}(\prod_{tm \in d_n}q_{mk})(\prod_{tm \in d_n}1-q_{mk})}$$

The actual computation as implemented by taking the sum of the log probabilities as opposed to the products of the probabilities themselves. This is necessary because once you are dealing with many terms, multiplying a lot of small numbers results in numeric underflow. Also, in order to prevent taking the log(0), which will happen in the first iteration, a very small number, $\epsilon$, is added to the probability before taking the log. So we end up with:

$$\tag{1a} \frac{\alpha_{k}e^{\left(\sum_{tm \in d_n}log(q_{mk}+\epsilon) + \sum_{tm \notin d_n}log(1-q_{mk}+\epsilon) \right)}}{\sum_{k}\alpha_{k} e^{\left(\sum_{tm \in d_n} \left(log(q_{mk}+\epsilon) + \sum_{tm \notin d_n}log(1-q_{mk}+\epsilon) \right) \right)}}$$

In the code below, a single pass is made to calculate the $r$ class soft assignments so that the terms are calculated only once so it may not be obvious what's going on.

_Note: the $\epsilon$ values are important for the algorithm to converge. If you take a straight calculation of the $r_{1,1}$ term in the first iteration you will end up with a divide by zero using the original equations. By including the $\epsilon$ term this is avoided and you'll see that the result is very close to 1_

**M Step**

$$\tag{2} q_{mk} = \frac{\sum_{n=1}^{N}r_{nk}I(t_m \in d_n)}{\sum_{i=1}^{N}r_{nk}}$$

$I(t_{m} \in d_{n}) = 1$ if term is an element of document n and 0 otherwise.

Finally, priors are updated per iteration as:

$$\tag{3} \alpha_k = \frac{\sum_{n=1}^{N}r_{nk}}{N}$$




In [1]:
# save this to a file - it will be handy for MR testing later
with open('test.txt','w') as outfile:
    outfile.write('hot chocolate cocoa beans\n')
    outfile.write('cocoa ghana africa\n')
    outfile.write('beans harvest ghana\n')
    outfile.write('cocoa butter\n')
    outfile.write('butter truffles\n')
    outfile.write('sweet chocolate\n')
    outfile.write('sweet sugar\n')
    outfile.write('sugar cane brazil\n')
    outfile.write('sweet sugar beet\n')
    outfile.write('sweet cake icing\n')
    outfile.write('cake black forest\n')

In [2]:
import re
from math import log
import numpy as np

# read the documents. Each document consists of a list of words.
documents = []
with open('test.txt', 'r') as docfile:
    for line in docfile.readlines():
        documents.append(re.split(' ', line.strip()))

classes = 2
r = [[None] * len(documents), [None] * len(documents)]

# set our initial conditions (r_6,1 = 1.0 and r_7,1 = 0.0 and the converse for the other class)
r[0][5] = r[1][6] = 1.0
r[0][6] = r[1][5] = 0.0

# initialize the priors
alpha = [0.0] * classes

epsilon = 0.0001

# conditional term probabilities
qm = {}

# compute alphas - Equation 3
def compute_alphas(alphas):
    for k in range(classes):
        alphas[k] = sum([x for x in r[k] if x is not None])/len([x for x in r[k] if x is not None])


# compute inverted postings list
# this is handy for the computation of the qm's
def compute_postings():
    postings = {}
    for i in range(len(documents)):
        if r[0][i] is not None:
            for word in documents[i]:
                if word not in postings:
                    postings[word] = [i]
                else:
                    if i not in postings[word]:
                        postings[word].append(i)
    return postings

# compute qm's - Equation 2
def compute_next_qms(qm):
    for k in range(classes):
        for i in range(len(r[k])):
            if r[k][i] is not None:
                for word in documents[i]:
                    if word not in qm:
                        qm[word] = {k: sum([r[k][j] for j in postings[word]]) / \
                                    sum([x for x in r[k] if x is not None])}
                    else:
                        qm[word][k] = sum([r[k][j] for j in postings[word]]) / \
                                    sum([x for x in r[k] if x is not None])
    

# compute next iteration of r's - Equation 1a
# note - need to do log(probability)
def compute_next_r(rs):
    for i in range(len(documents)):
        vocab_words_in_doc = []
        p = np.zeros((classes, 2))
        
        # find all vocab words in the doc. Note: there may be none.
        for word in documents[i]:
            if word in qm:
                vocab_words_in_doc.append(word)
                for k in range(classes):
                    p[k][0] += np.log(qm[word][k] + epsilon)
        if len(vocab_words_in_doc) > 0:
            # find all vocab words not in doc for all classes at the same time
            for word in qm:
                if word not in vocab_words_in_doc:
                    for k in range(classes):
                        p[k][1] += np.log(1-qm[word][k] + epsilon)
                        
            # compute the denominator of Equation 1a for all classes
            denom = 0.0
            for k in range(classes):
                denom += alpha[k]*np.exp(p[k][0]+p[k][1])
                
            # compute the new r of Equation 1a for all classes
            for k in range(classes):
                rs[k][i] = alpha[k]*np.exp(p[k][0]+p[k][1])/denom

        else:
            # set to prior in case of no information
            for k in range(classes):
                rs[k][i] = alpha[k]

# iterate
for _ in range(25):
    compute_alphas(alpha)
    postings = compute_postings()
    compute_next_qms(qm)
    compute_next_r(r)

In [3]:
# if you take out rounding you can see that these values are not really exact
np.around(r, 3)

array([[ 1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.]])